In [18]:
from unsloth import FastLanguageModel

max_seq_length = 1248 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_type = "gemma" # llama, phi-3, gemma

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "gsarti/gemma-2-2b-rebus-solver-adapters", # MODEL OR ADAPTER FOLDER
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.3.19: Fast Gemma2 patching. Transformers: 4.51.3.
   \\   /|    Tesla V100-PCIE-32GB. Num GPUs = 1. Max memory: 31.733 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.19 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [19]:
from datasets import load_dataset

eval_dataset = load_dataset('gsarti/eureka-rebus', 'llm_sft', data_files=["id_test.jsonl", "ood_test.jsonl"], split = "train")

In [20]:
# Single-example generation

stop_token_id = model.config.eos_token_id
if model_type == "gemma":
    stop_token = "<|eot_id|>"
    stop_token_id = tokenizer.encode(stop_token)[0]

In [21]:
from tqdm import tqdm

if model_type == "llama":
    tokenizer.padding_side = "right"
elif model_type in ("phi-3", "gemma"):
    tokenizer.padding_side = "left"

In [22]:
import re

regex_word_guess = '- \[.* = (.*)'
regex_firstpass = 'Prima lettura: (.*)'
regex_solution_word = "\d+ = (.*)"
regex_solution = "Soluzione: (.*)"

def parse_generation(data):
    try:
        word_guesses = ";".join(re.findall(regex_word_guess, data))
    except:
        word_guesses = ""
    try:
        first_pass = re.findall(regex_firstpass, data)[0]
    except:
        first_pass = ""
    try:
        solution_words = ";".join(re.findall(regex_solution_word, data))
    except:
        solution_words = ""
    try:
        solution = re.findall(regex_solution, data)[0]
    except:
        solution = ""
    
    return {
        "word_guesses": word_guesses,
        "first_pass": first_pass,
        "solution_words": solution_words,
        "solution": solution,
    }


In [23]:
# eval_dataset['conversations'][5][0]['value']

In [24]:
# parse_generation(eval_dataset['conversations'][5][1]['value'])

In [25]:
# inputs = tokenizer(eval_dataset['conversations'][5][0]['value'], return_tensors="pt")["input_ids"].to('cuda:0')
# outputs = model.generate(input_ids = inputs, max_new_tokens = 500, use_cache = True, eos_token_id = stop_token_id)
# model_generations = tokenizer.batch_decode(outputs)
# print(model_generations[0])

In [26]:
# from unsloth import FastLanguageModel

# max_seq_length = 1248 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# model_type = "gemma" # llama, phi-3, gemma

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "gsarti/gemma-2-2b-rebus-solver-fp16", # MODEL OR ADAPTER FOLDER
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )
# FastLanguageModel.for_inference(model)

# from datasets import load_dataset

# eval_dataset = load_dataset('gsarti/eureka-rebus', 'llm_sft', data_files=["id_test.jsonl", "ood_test.jsonl"], split = "train")

# # Single-example generation

# stop_token_id = model.config.eos_token_id
# if model_type == "gemma":
#     stop_token = "<|eot_id|>"
#     stop_token_id = tokenizer.encode(stop_token)[0]

# ex_idx = 5
# example = eval_dataset[ex_idx]["conversations"][0]
# inputs = tokenizer.apply_chat_template(
#     [
#         {"role": "user", "content": example["value"]}
#     ],
#     add_generation_prompt=True,
#     return_tensors = "pt",
#     padding=True,
#     truncation=True,
# )
# inputs = inputs.to('cuda:0')
# outputs = model.generate(input_ids = inputs, max_new_tokens = 500, use_cache = True, eos_token_id = stop_token_id)
# model_generations = tokenizer.batch_decode(outputs)
# print(model_generations[0])



# import re

# regex_word_guess = '- \[.* = (.*)'
# regex_firstpass = 'Prima lettura: (.*)'
# regex_solution_word = "\d+ = (.*)"
# regex_solution = "Soluzione: (.*)"

# def parse_generation(ex_idx, ex):
#     try:
#         word_guesses = ";".join(re.findall(regex_word_guess, ex))
#     except:
#         word_guesses = ""
#     try:
#         first_pass = re.findall(regex_firstpass, ex)[0]
#     except:
#         first_pass = ""
#     try:
#         solution_words = ";".join(re.findall(regex_solution_word, ex))
#     except:
#         solution_words = ""
#     try:
#         solution = re.findall(regex_solution, ex)[0]
#     except:
#         solution = ""
#     return {
#         "idx": ex_idx,
#         "word_guesses": word_guesses,
#         "first_pass": first_pass,
#         "solution_words": solution_words,
#         "solution": solution,
#     }




# import pandas as pd
# from tqdm import tqdm

# # Create gold parsed outputs

# results = []
# for ex_idx, ex in tqdm(enumerate(eval_dataset), total=len(eval_dataset)):
#     gold_output = ex["conversations"][1]["value"]
#     parsed_output = parse_generation(ex_idx, gold_output)
#     results.append(parsed_output)

In [27]:
# from tqdm import tqdm

# if model_type == "llama":
#     tokenizer.padding_side = "right"
# elif model_type in ("phi-3", "gemma"):
#     tokenizer.padding_side = "left"

# results = []
# batch_size = 134
# for i in tqdm(range(0, len(eval_dataset), batch_size), total=len(eval_dataset)//batch_size):
#     batch = eval_dataset[i:i+batch_size]
#     print(batch['conversations'][0])
#     if model_type == "llama":
#         input = [[{"role": "user", "content": example[0]["value"]}] for example in batch["conversations"]]
    # elif model_type == "phi-3":
        # print(batch['conversation'])
        # input = [[item[0]] for item in batch["conversations"]]

    # inputs = tokenizer.apply_chat_template(
    #     input,
    #     tokenize = True,
    #     add_generation_prompt = True, # Must add for generation
    #     return_tensors = "pt",
    #     padding=True,
    #     truncation=True,
    #     return_dict=True
    # )
    # inputs = {k: v.to("cuda") for k, v in inputs.items()}
    # outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = True, eos_token_id = stop_token_id)

    # model_generations = tokenizer.batch_decode(outputs)
    # for ex_idx, ex in enumerate(model_generations):
    #     out_dic = parse_generation(ex_idx + i, ex)
    #     if i == 0 and ex_idx <= 5:
    #         print(ex)
    #         print(out_dic)
    #     results.append(out_dic)

# qui

In [28]:
# Single-example generation

stop_token_id = model.config.eos_token_id
if model_type == "gemma":
    stop_token = "<|eot_id|>"
    stop_token_id = tokenizer.encode(stop_token)[0]

ex_idx = 5
example = eval_dataset[ex_idx]["conversations"][0]
inputs = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": example["value"]}
    ],
    add_generation_prompt=True,
    return_tensors = "pt",
    padding=True,
    truncation=True,
)
inputs = inputs.to('cuda:0')
outputs = model.generate(input_ids = inputs, max_new_tokens = 500, use_cache = True, eos_token_id = stop_token_id)
model_generations = tokenizer.batch_decode(outputs)
parse_generation(model_generations[0])

{'word_guesses': 'pire;cent',
 'first_pass': 'TEM pire cent I',
 'solution_words': 'Tempi;recenti',
 'solution': 'Tempi recenti'}

In [29]:
len(eval_dataset)

1998

In [30]:
eval_dataset

Dataset({
    features: ['conversations'],
    num_rows: 1998
})

In [31]:
# Seleziona un subset di dimensione 10
subset = eval_dataset.select(range(10))

# Verifica il subset
print(subset)

Dataset({
    features: ['conversations'],
    num_rows: 10
})


In [32]:
results = []

for ex_idx in tqdm(range(len(subset)), total=len(subset)):
    # Ottieni il primo esempio di conversazione (senza batch)
    examples = subset[ex_idx]
    prompts = examples['conversations'][0]

    inputs = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": prompts["value"]}
    ],
    add_generation_prompt=True,
    return_tensors = "pt",
    padding=True,
    truncation=True,
    )
    inputs = inputs.to('cuda:0')
    
    # print(example)  # Puoi usare questo per controllare il contenuto dell'esempio

    # input = [[item[0]] for item in examples]
    
    # # A questo punto puoi fare inferenza su `input`, ad esempio:
    # inputs = tokenizer.apply_chat_template(
    #     input,
    #     tokenize=True,
    #     add_generation_prompt=True,  # Aggiungi il prompt per la generazione
    #     return_tensors="pt",
    #     padding=True,
    #     truncation=True,
    #     return_dict=True
    # )

    # # Inferenza sul modello
    outputs = model.generate(inputs, max_new_tokens=500, use_cache=True, eos_token_id=stop_token_id)
    model_generations = tokenizer.batch_decode(outputs)
    
    # # Aggiungi il risultato alla lista dei risultati
    results.append(parse_generation(model_generations[0]))
    
    # # Opzionalmente, puoi fare qualcosa con i risultati, come stamparli:
    print(model_generations[0])


 10%|████████                                                                        | 1/10 [00:21<03:11, 21.26s/it]

<bos><start_of_turn>user
Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.

Rebus: [Sono pari nell' odio] C CI [Altro nome dei gicheri] NF [Se scappa, va in esilio] SC [Le ha l'armadio]
Chiave di lettura: 6 12<end_of_turn>
<start_of_turn>model
Procediamo alla risoluzione del rebus passo per passo:
- [Sono pari nell' odio] = do
- C C I = C C I
- [Altro nome dei gicheri] = ari
- N F = N F
- [Se scappa, va in esilio] = re
- S C = S C
- [Le ha l'armadio] = ante

Prima lettura: do C CI ari NF re SC ante

Ora componiamo la soluzione seguendo la chiave risolutiva:
6 = Doccia
12 = rinfrescante

Soluzione: Doccia rinfrescante
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>

 20%|████████████████                                                                | 2/10 [00:42<02:49, 21.24s/it]

<bos><start_of_turn>user
Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.

Rebus: A [Un essere strisciante] M [Costosi ornamenti] [È a destra o a sinistra] C UN [Audace, ardimentosa]
Chiave di lettura: 4 7 8<end_of_turn>
<start_of_turn>model
Procediamo alla risoluzione del rebus passo per passo:
- A = A
- [Un essere strisciante] = verme
- M = M
- [Costosi ornamenti] = ori
- [È a destra o a sinistra] = ala
- C U N = C U N
- [Audace, ardimentosa] = osa

Prima lettura: A verme M ori ala C UN osa

Ora componiamo la soluzione seguendo la chiave risolutiva:
4 = Aver
7 = memoria
8 = lacunosa

Soluzione: Aver memoria lacunosa
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id

 30%|████████████████████████                                                        | 3/10 [01:03<02:28, 21.24s/it]

...

<bos><start_of_turn>user
Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.

Rebus: TR [La compagna di Adamo] L [Sono fatti per il riposo] M [Sono ambiti alle Olimpiadi]
Chiave di lettura: 3 8 4<end_of_turn>
<start_of_turn>model
Procediamo alla risoluzione del rebus passo per passo:
- T R = T R
- [La compagna di Adamo] = Eva
- L = L
- [Sono fatti per il riposo] = letti
- M = M
- [Sono ambiti alle Olimpiadi] = ori

Prima lettura: TR Eva L letti M ori

Ora componiamo la soluzione seguendo la chiave risolutiva:
3 = Tre
8 = valletti
4 = mori

Soluzione: Tre valletti mori
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_i

 90%|████████████████████████████████████████████████████████████████████████        | 9/10 [03:11<00:21, 21.25s/it]

<bos><start_of_turn>user
Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.

Rebus: R [Altare o pappagallo] I [Preposizione di accompagnamento] A
Chiave di lettura: 4 5<end_of_turn>
<start_of_turn>model
Procediamo alla risoluzione del rebus passo per passo:
- R = R
- [Altare o pappagallo] = ara
- I = I
- [Preposizione di accompagnamento] = con
- A = A

Prima lettura: R ara I con A

Ora componiamo la soluzione seguendo la chiave risolutiva:
4 = Rara
5 = icona

Soluzione: Rara icona
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_i

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [03:32<00:00, 21.24s/it]

<bos><start_of_turn>user
Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.

Rebus: [Preposizione intermedia] [Popolare Ombretta dello spettacolo] F A [Questi]
Chiave di lettura: 8 6<end_of_turn>
<start_of_turn>model
Procediamo alla risoluzione del rebus passo per passo:
- [Preposizione intermedia] = tra
- [Popolare Ombretta dello spettacolo] = colli
- F A = F A
- [Questi] = tali

Prima lettura: tra colli F A tali

Ora componiamo la soluzione seguendo la chiave risolutiva:
8 = Tracolli
6 = fatali

Soluzione: Tracolli fatali
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_id|>
<|eot_i

In [33]:
results

[{'word_guesses': 'do;ari;re;ante',
  'first_pass': 'do C CI ari NF re SC ante',
  'solution_words': 'Doccia;rinfrescante',
  'solution': 'Doccia rinfrescante'},
 {'word_guesses': 'verme;ori;ala;osa',
  'first_pass': 'A verme M ori ala C UN osa',
  'solution_words': 'Aver;memoria;lacunosa',
  'solution': 'Aver memoria lacunosa'},
 {'word_guesses': 'mazzo;eco;raggio',
  'first_pass': 'A mazzo N eco raggio SA',
  'solution_words': 'Amazzone;coraggiosa',
  'solution': 'Amazzone coraggiosa'},
 {'word_guesses': 're;storico;sci;totale',
  'first_pass': 'P re TE storico NO sci U totale',
  'solution_words': 'Pretesto;riconosciuto;tale',
  'solution': 'Pretesto riconosciuto tale'},
 {'word_guesses': 'orme;destre;ori;ore',
  'first_pass': 'N orme destre M ori G ore',
  'solution_words': 'Norme;d;estremo;rigore',
  'solution': "Norme d'estremo rigore"},
 {'word_guesses': 'pire;cent',
  'first_pass': 'TEM pire cent I',
  'solution_words': 'Tempi;recenti',
  'solution': 'Tempi recenti'},
 {'word_g